In [1]:
import google.generativeai as genai
import os
import pandas as pd
import time
from tqdm import tqdm
tqdm.pandas()

/opt/anaconda3/envs/genai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
# Configure with your Gemini API key
genai.configure(api_key="<YOUR KEY HERE>")

# Initialize Gemini 2.0 Flash model
model = genai.GenerativeModel('gemini-2.0-flash')

# --- Prompt Template ---
template = """
You are about to be given a cricket commentary, perform the tasks listed on your own:

Commentary: {commentary}

Tasks:
1. Make batsman scorecard containing batsman,Runs,Balls_Faced,Fours,Sixes,S/R,dismissals.
2. Make bowler scorecard containing bowler,Balls,Runs_Given,Wickets,Overs,Maidens.
3. Write the answers in the form of JSON format
{{'batsman':[batsman1,batsman2,...],'Runs':[batsman1_runs,batsman2_runs,...],'Balls_Faced':[Balls1,Balls2,...],'Fours':[fours1,fours2,...],'Sixes':[sizes1,sizes2,...],'S/R':[strikerate1,strikerate2,...],'dismissals':[dismissal1,dismissal2,...]}}
{{'bowler':[bowler1,bowler2,...],'Balls':[Balls_bowled_baller1,Balls_bowled_baller2,...],'Runs_Given':[Runs_given_bowler1,Runs_given_bowler2,...],'Wickets':[wicket_taken_bowler1,wicket_taken_bowler2,...],'Overs':[overs_bolwed_bowler_1,overs_bolwed_bowler_2,...],'Maidens':[maiden_bowler_1,maiden_bowler_2,...]}}

Let's think step by step.
"""

In [2]:
curr_path = '/Users/ritamupadhyay/Documents/Coral/table_gen'
os.chdir(curr_path)

In [3]:
table_path = os.path.join(curr_path,"data_commentaries/odi/odi_complete.csv")

In [4]:
df = pd.read_csv(table_path)

In [59]:
df

,commentaries,batsman_gnd_truth,bowler_gnd_truth,ref_table,gemini_2_f_zscot,batsman_pred,bowler_pred
0,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to analyze the commentary and ...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
2,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
3,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
4,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"```json\n{\n ""batsman"": [\n ""Tamim Iqbal"",...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
...,...,...,...,...,...,...,...
1291,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,Here's the breakdown of the cricket commentary...,data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1292,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"```json\n{\n""batsman"": [""Tamim Iqbal"", ""Mohamm...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1293,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1294,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to analyze the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...


In [10]:
df['gemini_2_f'] = ""

In [39]:
df

,commentaries,batsman_gnd_truth,bowler_gnd_truth,ref_table,gemini_2_f_zscot,batsman_pred,bowler_pred
0,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to analyze the commentary and ...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
2,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
3,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
4,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"```json\n{\n ""batsman"": [\n ""Tamim Iqbal"",...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
...,...,...,...,...,...,...,...
1291,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,Here's the breakdown of the cricket commentary...,data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1292,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"```json\n{\n""batsman"": [""Tamim Iqbal"", ""Mohamm...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1293,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1294,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to analyze the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...


In [40]:
def get_model_response(commentary):
    prompt = template.format(commentary=commentary)
    # Generate response
    response = model.generate_content(prompt)

    return response.text

In [20]:
counter = 1
for idx,row in tqdm(df.iterrows()):
    commentary = row['commentaries']
    if row['gemini_2_f']!="":
        continue
    try:
        df.at[idx,'gemini_2_f'] = get_model_response(commentary)
        if counter % 8 == 0:
            time.sleep(30)
        counter += 1
    except:
        continue

0it [00:00, ?it/s]

1296it [44:38,  2.07s/it]


In [60]:
## rerun the defective ones
for idx,row in tqdm(df[df['bowler_pred'].isna()==True].iterrows()):
    commentary = row['commentaries']
    try:
        df.at[idx,'gemini_2_f_zscot'] = get_model_response(commentary)
        time.sleep(10)
    except:
        continue

4it [00:45, 11.41s/it]


In [61]:
df

,commentaries,batsman_gnd_truth,bowler_gnd_truth,ref_table,gemini_2_f_zscot,batsman_pred,bowler_pred
0,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to analyze the commentary and ...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
2,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
3,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
4,"Muzarabani to Tamim, no run fullish ball on ...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"```json\n{\n ""batsman"": [\n ""Tamim Iqbal"",...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
...,...,...,...,...,...,...,...
1291,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,Here's the breakdown of the cricket commentary...,data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1292,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"```json\n{\n""batsman"": [""Tamim Iqbal"", ""Mohamm...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1293,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to process the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...
1294,"Dhananjaya to Tamim, no run length ball on o...",data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/ground_truth_tables/Ball...,data_commentaries/odi/tables/Ball by Ball Comm...,"Okay, I'm ready to analyze the cricket comment...",data_commentaries/odi/pred_tables/Ball by Ball...,data_commentaries/odi/pred_tables/Ball by Ball...


In [62]:
df.to_csv(table_path,index=False)

In [30]:
commentary = df.iloc[342]['commentaries']
# Format the prompt
prompt = template.format(commentary=commentary)
# Generate response
response = model.generate_content(prompt)

# Print the result
print(response.text)

Okay, I'm ready to process the cricket commentary and generate the requested JSON output. Here's the breakdown of my approach:

**1. Data Extraction:**

*   I'll meticulously scan the commentary, identifying each batsman's score, balls faced, boundaries (4s and 6s), and dismissal information (if any).
*   I'll also track each bowler's deliveries, runs conceded, wickets taken, and identify any maidens bowled.

**2. Calculation:**

*   I will calculate the strike rate for each batsman using the formula: (Runs / Balls Faced) \* 100.
*   Overs bowled will be derived from the balls bowled by each bowler (Balls / 6). This may result in floating-point numbers (e.g., 9.3 for 9 overs and 3 balls).

**3. JSON Formatting:**

*   I will then structure the extracted and calculated data into two JSON dictionaries, one for batsman stats and another for bowler stats, adhering to the exact format you provided.

```json
{
    "batsman": [
        "Avishka Fernando",
        "Bhanuka",
        "Rajapaksa

In [31]:
df.at[342,'gemini_2_f'] = response.text

In [33]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Okay, I'm ready to process the cricket commentary and produce the JSON output.\n\n**Here's the breakdown of how I'll approach this:**\n\n1.  **Data Extraction:** I'll read the commentary and identify each ball bowled, who bowled it, who faced it, and the outcome (runs scored, wicket taken, wide, no-ball, etc.).\n\n2.  **Scorecard Calculation:** I will initialize the scorecard dictionaries. As I parse the commentary, I'll increment the relevant stats:\n    *   **Batsman:** Runs, Balls Faced, Fours, Sixes\n    *   **Bowler:** Balls, Runs Given, Wickets, Maidens.\n\n3.  **Overs and Maidens Calculation**: Maidens will be tracked during the commentary, Overs will be calculated from balls.\n\n4.  **JSON Conversion:** Finally, I'll structure the calculated data int

In [34]:
print(pd.read_csv(df.iloc[0]['batsman_gnd_truth']))

      batsman  Runs  Balls_Faced  Fours  Sixes    S/R dismissal
0  Litton Das   4.0           16      1      0  25.00   Not Out
1       Tamim   9.0           14      2      0  64.29   Not Out


In [31]:
for _,row in df.sample(20).iterrows():
    print(row['gemini_2_f'])
    print(pd.read_csv(row['batsman_gnd_truth']))
    print(pd.read_csv(row['bowler_gnd_truth']))

Okay, I'm ready to process the commentary and provide the batsman and bowler scorecards in JSON format. Here's how I'll approach this:

**1. Data Extraction and Processing:**

*   **Batsman Data:** I'll scan the commentary for each batsman (Rohit Sharma, Shubman Gill, Virat Kohli, Shreyas Iyer). I will identify runs scored, boundaries (4s and 6s), balls faced, and dismissals.
*   **Bowler Data:** I'll identify each bowler (Taskin Ahmed, Mustafizur Rahman, Tanzim Hasan Sakib, Mehidy Hasan Miraz, Rishad Hossain). I will track balls bowled, runs conceded, wickets taken, and Maidens overs bowled. To calculate maidens I have to check the commentary ball by ball.

**2. Calculations:**

*   **Strike Rate (Batsmen):** Calculate strike rate using the formula: (Runs / Balls Faced) \* 100.
*   **Overs (Bowlers):** Calculate overs bowled from balls bowled (Overs = Balls / 6). The remainder will be the number of balls bowled in the last over.
*   **Maidens(Bowler):** Calculating the Maiden overs bo

In [24]:
print(pd.read_csv(df.iloc[0]['bowler_gnd_truth']))

       bowler  Balls  Runs_Given  Wickets  Overs  Maidens
0     Chatara     12         5.0        0    2.0        1
1  Muzarabani     18        10.0        0    3.0        0


In [25]:
df['commentaries'].nunique()

1296

In [26]:
response.text

"Okay, I will analyze the commentary and create the requested scorecards.\n\n**Step 1: Analyze the Commentary**\n\nI need to track each ball bowled, who bowled it, who faced it, the runs scored, and any dismissals.  The commentary doesn't explicitly state any dismissals, so I will assume both batsmen are not out. Also, 6's were never scored so they will remain 0.\n\n**Step 2: Create Batsman Scorecard**\n\n*   **Batsman:** Tamim, Litton Das\n*   **Runs:** Need to sum the runs scored by each batsman.\n*   **Balls Faced:**  Count the number of balls faced by each batsman.\n*   **Fours:** Count the number of fours hit by each batsman.\n*   **Sixes:** There are no sixes.\n*   **S/R:** (Runs / Balls Faced) * 100\n*   **Dismissals:** 0 for both as there are no dismissals in the commentary.\n\n**Step 3: Create Bowler Scorecard**\n\n*   **Bowler:** Muzarabani, Chatara\n*   **Balls:** Count the number of balls bowled by each bowler. Note that wides are included in runs but not balls bowled.\n*  

In [13]:
ind = 51
print(pd.read_csv(df.iloc[ind]['bowler_gnd_truth']))
print(pd.read_csv(df.iloc[ind]['bowler_pred']))
print("\n\n")
print(pd.read_csv(df.iloc[ind]['batsman_gnd_truth']))
print(pd.read_csv(df.iloc[ind]['batsman_pred']))

                bowler  Balls  Runs_Given  Wickets  Overs  Maidens
0               Faheem     18        16.0        0    3.0        0
1           Haris Rauf     18        21.0        0    3.0        0
2            Hasan Ali     24        33.0        0    4.0        0
3  Shaheen Shah Afridi     30        22.0        1    5.0        0
                bowler  Balls  Runs_Given  Wickets  Overs  Maidens
0  Shaheen Shah Afridi     42          22        1    7.0        2
1            Hasan Ali     36          29        1    6.0        0
2           Haris Rauf     30          29        0    5.0        0
3               Faheem     30          22        0    5.0        0



   batsman  Runs  Balls_Faced  Fours  Sixes     S/R  \
0  Crawley  37.0           33      5      0  112.12   
1    Malan  43.0           45      6      0   95.56   
2     Salt   7.0           13      1      0   53.85   

                                dismissal  
0                                 Not Out  
1                 

In [18]:
df_temp = pd.read_csv(df.iloc[51]['ref_table'])

In [23]:
df.iloc[51]['batsman_pred']

'data_commentaries/odi/pred_tables/Ball by Ball Commentary & Live Score - ENG vs PAK, 1st ODI_batsman_scorecard_15.csv'

In [26]:
df.columns

Index(['commentaries', 'batsman_gnd_truth', 'bowler_gnd_truth', 'ref_table',
       'gemini_2_f_zscot', 'batsman_pred', 'bowler_pred'],
      dtype='object')